In [1]:
#We import these libraries
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime


In [37]:
#an check if we have a functioning internet connection
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        return True
    except:
        conn.close()
        return False

In [38]:
check_internet()

True

In [39]:
# this is a function that is taking in the query url
# and a path for the json and the csv to be stored
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    #we get the stock id by retrieving it from the query url as the string between symbol= and &period 
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    # if the csv already exists, we update the data simpy 
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return

    #otherwise we remove the json and add a new json dump
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            #for each entry in the parsed json we restructure it into lists
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))
            
            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            #that we put into an array that we write into a csv

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [40]:
#we write the path for the folder structure which is based on different os dependent notations
json_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"json"+os.sep
csv_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"csv"+os.sep


In [41]:
# here we create directories if they do not exist
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)


In [42]:
#we define some predetermine variables
period1 = -1325583000
period2 = 9999999999
interval = "1d"


In [43]:
#and our custom dictionary of dax 40 companies and their stock id
full_ticker_dict={
"Dow Jones":'^DJI',
    "S&P 500":"^GSPC",
    "Nasdaq":"^NDX",
    "Russel":"^RUT"
 }

In [44]:
query_urls=[]
#now we go through all the ticker stock ids and create urls
for ticker in full_ticker_dict.values():
    query_urls.append("https://query1.finance.yahoo.com/v8/finance/chart/"+ticker+"?symbol="+ticker+"&period1=0&period2=9999999999&interval=1d&includePrePost=true&events=div%2Csplit")

In [45]:
#and then we go through all urls in a pool process
with Pool(processes=len(query_urls)) as pool:
    pool.starmap(get_historic_price, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("All downloads completed !")

<<<  Historical data of ^DJI already exists, Updating data...
<<<  Historical data of ^NDX already exists, Updating data...
<<<  Historical data of ^GSPC already exists, Updating data...
<<<  Historical data of ^RUT already exists, Updating data...
>>>  Historical data of ^DJI exists but has no trading data
>>>  Historical data of ^NDX exists but has no trading data
>>>  Historical data of ^RUT exists but has no trading data
>>>  Historical data of ^GSPC exists but has no trading data
All downloads completed !


In [46]:
# we lastly create a translate df to write from the stock id the entire name of the stock
df_translate=pd.DataFrame(full_ticker_dict,index=[0]).T.reset_index()
df_translate.columns=["name","shortener"]
df_translate

,name,shortener
0,Dow Jones,^DJI
1,S&P 500,^GSPC
2,Nasdaq,^NDX
3,Russel,^RUT


In [47]:
df_translate.to_csv("../data/translate.csv",index=False)

In [226]:
# this proves it has worked
shortener_name=df_translate[df_translate["name"]=="S&P 500"].shortener.values[0]
SP_yahoo=pd.read_csv("../historic_data/csv/"+shortener_name+".csv")
SP_yahoo=SP_yahoo[["Date","Close"]]
SP_yahoo["Date"]=pd.to_datetime(SP_yahoo["Date"],format="%d-%m-%Y")
SP_yahoo

,Date,Close
0,1970-01-02,93.000000
1,1970-01-05,93.459999
2,1970-01-06,92.820000
3,1970-01-07,92.629997
4,1970-01-08,92.680000
...,...,...
13822,2024-10-22,5851.200195
13823,2024-10-23,5797.419922
13824,2024-10-24,5809.859863
13825,2024-10-25,5808.120117


In [227]:
further_historic_sp=pd.read_csv("../data/SP500.csv")
further_historic_sp=further_historic_sp[["Date","Close"]]
further_historic_sp["Date"]=pd.to_datetime(further_historic_sp["Date"])
merged_sp=pd.concat([further_historic_sp,SP_yahoo]).drop_duplicates(keep="first")
merged_sp

,Date,Close
0,1927-12-30,17.660000
1,1928-01-03,17.760000
2,1928-01-04,17.719999
3,1928-01-05,17.549999
4,1928-01-06,17.660000
...,...,...
13822,2024-10-22,5851.200195
13823,2024-10-23,5797.419922
13824,2024-10-24,5809.859863
13825,2024-10-25,5808.120117


In [228]:
# this proves it has worked
shortener_name=df_translate[df_translate["name"]=="Dow Jones"].shortener.values[0]
dow_yahoo=pd.read_csv("../historic_data/csv/"+shortener_name+".csv")
dow_yahoo=dow_yahoo[["Date","Close"]]
dow_yahoo["Date"]=pd.to_datetime(dow_yahoo["Date"],format="%d-%m-%Y")
dow_yahoo

,Date,Close
0,1992-01-02,3172.399902
1,1992-01-03,3201.500000
2,1992-01-06,3200.100098
3,1992-01-07,3204.800049
4,1992-01-08,3203.899902
...,...,...
8262,2024-10-22,42924.890625
8263,2024-10-23,42514.949219
8264,2024-10-24,42374.359375
8265,2024-10-25,42114.398438


In [229]:
further_historic_dow=pd.read_csv("../data/DJA.csv")
further_historic_dow=further_historic_dow.rename(columns={"Unnamed: 0":"Date"})
further_historic_dow=further_historic_dow[["Date","Close"]]
further_historic_dow["Date"]=pd.to_datetime(further_historic_dow["Date"])
merged_dow=pd.concat([further_historic_dow,dow_yahoo]).drop_duplicates(keep="first")
merged_dow

,Date,Close
0,1885-05-02,31.435000
1,1885-05-04,30.661400
2,1885-05-05,30.508700
3,1885-05-06,30.824000
4,1885-05-07,30.503800
...,...,...
8262,2024-10-22,42924.890625
8263,2024-10-23,42514.949219
8264,2024-10-24,42374.359375
8265,2024-10-25,42114.398438


In [230]:
import datetime


In [231]:
election_days=[]
for year in range(2024,1855,-4):
    print(year)
    weekday_number=0
    day=2
    while weekday_number!=1:
        start_datetime = datetime.datetime(year,11,day,00,00,00)
        weekday_number = start_datetime.date().weekday()
        day+=1

    election_days.append(start_datetime)
election_df=pd.DataFrame({"election_day":election_days})
election_df

2024
2020
2016
2012
2008
2004
2000
1996
1992
1988
1984
1980
1976
1972
1968
1964
1960
1956
1952
1948
1944
1940
1936
1932
1928
1924
1920
1916
1912
1908
1904
1900
1896
1892
1888
1884
1880
1876
1872
1868
1864
1860
1856


,election_day
0,2024-11-05
1,2020-11-03
2,2016-11-08
3,2012-11-06
4,2008-11-04
5,2004-11-02
6,2000-11-07
7,1996-11-05
8,1992-11-03
9,1988-11-08


In [232]:
Unified=pd.read_csv("../data/Unified.csv")
Unified["Year"]=Unified["Congress"].str[-10:-6].astype(int)-1
Unified=Unified[["Year","Party Government","Presidency"]]
Unified["Year"]=Unified["Year"].astype(str)
Unified["Presidency"]=Unified["Presidency"].str[:3]
Unified

,Year,Party Government,Presidency
0,1856,Unified,Dem
1,1858,Divided,Dem
2,1860,Unified,Rep
3,1862,Unified,Rep
4,1864,Unified/ Divided2,Rep
...,...,...,...
79,2014,Divided,Dem
80,2016,Unified,Rep
81,2018,Divided,Rep
82,2020,Unified,Dem


In [234]:
election_df['Year'] = election_df['election_day'].astype(str).str[:4]

election_df_unified=pd.merge(election_df,Unified,left_on="Year",right_on="Year")
election_df_unified


,election_day,Year,Party Government,Presidency
0,2020-11-03,2020,Unified,Dem
1,2016-11-08,2016,Unified,Rep
2,2012-11-06,2012,Divided,Dem
3,2008-11-04,2008,Unified,Dem
4,2004-11-02,2004,Unified,Rep
5,2000-11-07,2000,Unified / Divided,Rep
6,1996-11-05,1996,Divided,Dem
7,1992-11-03,1992,Unified,Dem
8,1988-11-08,1988,Divided,Rep
9,1984-11-06,1984,Divided,Rep


In [235]:
election_df_unified=election_df_unified[election_df_unified["Year"].astype(int)>=1920]
election_df_unified

,election_day,Year,Party Government,Presidency
0,2020-11-03,2020,Unified,Dem
1,2016-11-08,2016,Unified,Rep
2,2012-11-06,2012,Divided,Dem
3,2008-11-04,2008,Unified,Dem
4,2004-11-02,2004,Unified,Rep
5,2000-11-07,2000,Unified / Divided,Rep
6,1996-11-05,1996,Divided,Dem
7,1992-11-03,1992,Unified,Dem
8,1988-11-08,1988,Divided,Rep
9,1984-11-06,1984,Divided,Rep


In [237]:
after_election_days=90
before_election_days=7
election_df_to_pass=election_df_unified
index_df=merged_dow

In [238]:
def comparison_df_calc(comparison_df,before_election_days,after_election_days):
    comparison_array=[]
    for index, row in comparison_df.iloc[:].iterrows():
        one_day_before=(row["Date"]-  pd.to_timedelta(1, unit='d'))
        time_before_comparison=(row["Date"]-  pd.to_timedelta(before_election_days, unit='d'))
        before_comparison_df=comparison_df[(comparison_df["Date"]<=one_day_before)&(comparison_df["Date"]>=time_before_comparison)]
        
        before_value=(before_comparison_df["Close"].mean())
    
        one_day_plus=(row["Date"]+  pd.to_timedelta(1, unit='d'))
        cut_off_date=(row["Date"]+  pd.to_timedelta(after_election_days, unit='d'))
    
        after_comparison_df=comparison_df[(comparison_df["Date"]>=one_day_plus)&(comparison_df["Date"]<=cut_off_date)]
        after_value=(after_comparison_df["Close"].mean())
        comparison_array.append([before_value,after_value])
    comparison_mean_df=pd.DataFrame(comparison_array,columns=["before","after"])
    comparison_mean_df["earn"]=(comparison_mean_df["after"]-comparison_mean_df["before"])*100/comparison_mean_df["before"]
    comparison_mean_value=(comparison_mean_df["earn"].mean())
    return comparison_mean_value

In [239]:
def election_investment(election_df, index_df,before_election_days, after_election_days):

    election_df['after_election_day'] = election_df['election_day'] +  pd.to_timedelta(1, unit='d')
    election_df['before_election_day'] = election_df['election_day'] -  pd.to_timedelta(1, unit='d')
    election_df['after_election_time'] = election_df['election_day'] +  pd.to_timedelta(after_election_days, unit='d')
    election_df['before_election_time'] = election_df['election_day'] -  pd.to_timedelta(before_election_days, unit='d')
    election_df['comparison_election_time'] = election_df['election_day'] -  pd.to_timedelta((365*4), unit='d')
        
    overall_df=[]
    
    for index, row in election_df.iloc[:].iterrows():
        year=row["Year"]
        presidency=row["Presidency"]
    
        print(year)
        after_t_start=(row["after_election_day"])
        after_t_end=(row["after_election_time"])
        before_t_start=(row["before_election_time"])
        before_t_end=(row["before_election_day"])
        comparison_t_start=(row["comparison_election_time"])
        comparison_t_end=(row["before_election_day"])
    
    
        after_index_df=index_df[(index_df["Date"]>=after_t_start)&(index_df["Date"]<=after_t_end)]
        before_index_df=index_df[(index_df["Date"]<=before_t_end)&(index_df["Date"]>=before_t_start)]
        mean_value_after=after_index_df["Close"].mean()
        mean_value_before=before_index_df["Close"].mean()
        mean_earn=(mean_value_after-mean_value_before)*100/mean_value_before
        comparison_df=index_df[(index_df["Date"]<=comparison_t_end)&(index_df["Date"]>=comparison_t_start)]
        comparison_mean_value=comparison_df_calc(comparison_df,before_election_days,after_election_days)
        overall_df.append([year,mean_earn,comparison_mean_value,presidency])
    overall_df_red=pd.DataFrame(overall_df,columns=["Year","Earn","Comp_Earn","Presidency"])
    overall_df_red=overall_df_red.dropna()
    return(overall_df_red)

In [240]:
result_dow=election_investment(election_df_to_pass, merged_dow,before_election_days, after_election_days)
result_dow

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_90020/2738988309.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_df['after_election_day'] = election_df['election_day'] +  pd.to_timedelta(1, unit='d')
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_90020/2738988309.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_df['before_election_day'] = election_df['election_day'] -  pd.to_timedelta(1, unit='d')
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_9002

2020
2016
2012
2008
2004
2000
1996
1992
1988
1984
1980
1976
1972
1968
1964
1960
1956
1952
1948
1944
1940
1936
1932
1928
1924
1920


,Year,Earn,Comp_Earn,Presidency
0,2020,12.327942,1.576060,Dem
1,2016,8.797910,1.189205,Rep
2,2012,0.864995,1.700283,Dem
3,2008,-7.462755,-0.297174,Dem
4,2004,5.647904,0.013346,Rep
5,2000,-2.287256,1.876127,Rep
6,1996,8.337170,2.153087,Dem
7,1992,0.965324,1.315882,Dem
8,1988,1.101081,2.224007,Rep
9,1984,-0.205662,0.962045,Rep


In [241]:
result_dow

,Year,Earn,Comp_Earn,Presidency
0,2020,12.327942,1.576060,Dem
1,2016,8.797910,1.189205,Rep
2,2012,0.864995,1.700283,Dem
3,2008,-7.462755,-0.297174,Dem
4,2004,5.647904,0.013346,Rep
5,2000,-2.287256,1.876127,Rep
6,1996,8.337170,2.153087,Dem
7,1992,0.965324,1.315882,Dem
8,1988,1.101081,2.224007,Rep
9,1984,-0.205662,0.962045,Rep


In [222]:
result_sp

,Year,Earn,Comp_Earn,Presidency
0,2020,11.493114,1.636536,Dem
1,2016,6.706115,1.454696,Rep
2,2012,1.260237,1.892105,Dem
3,2008,-8.522613,-0.506033,Dem
4,2004,5.358133,-0.434708,Rep
5,2000,-6.040684,2.326541,Rep
6,1996,6.800283,1.738290,Dem
7,1992,2.786764,1.532893,Dem
8,1988,0.519889,1.968810,Rep
9,1984,0.367943,0.906053,Rep


In [121]:
len(result_dow[result_dow["Earn"]>0])

18

In [167]:
outperform_dow=len(result_dow[result_dow["Earn"]>result_dow["Comp_Earn"]])
outperform_dow

14

In [168]:
underperform_dow=len(result_dow[result_dow["Earn"]<result_dow["Comp_Earn"]])
underperform_dow

12

In [242]:
result_dow=result_dow.sort_values(by="Year")
result_dow

,Year,Earn,Comp_Earn,Presidency
25,1920,-11.984196,-0.445074,Rep
24,1924,11.786562,1.174795,Rep
23,1928,14.152291,2.936769,Rep
22,1932,-0.169420,-3.684967,Dem
21,1936,3.349992,3.937843,Dem
20,1940,-2.228827,-0.658516,Dem
19,1944,2.560498,0.492580,Dem
18,1948,-6.092756,0.982512,Dem
17,1952,6.423294,1.476189,Rep
16,1956,-0.586767,1.940659,Rep


In [243]:
result_dow.to_csv("result_dow.csv",index=False)

In [245]:
result_sp=election_investment(election_df_to_pass, merged_sp,before_election_days, after_election_days)
result_sp

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_90020/2738988309.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_df['after_election_day'] = election_df['election_day'] +  pd.to_timedelta(1, unit='d')
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_90020/2738988309.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_df['before_election_day'] = election_df['election_day'] -  pd.to_timedelta(1, unit='d')
/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_9002

2020
2016
2012
2008
2004
2000
1996
1992
1988
1984
1980
1976
1972
1968
1964
1960
1956
1952
1948
1944
1940
1936
1932
1928
1924
1920


,Year,Earn,Comp_Earn,Presidency
0,2020,11.493114,1.636536,Dem
1,2016,6.706115,1.454696,Rep
2,2012,1.260237,1.892105,Dem
3,2008,-8.522613,-0.506033,Dem
4,2004,5.358133,-0.434708,Rep
5,2000,-6.040684,2.326541,Rep
6,1996,6.800283,1.738290,Dem
7,1992,2.786764,1.532893,Dem
8,1988,0.519889,1.968810,Rep
9,1984,0.367943,0.906053,Rep


In [246]:
result_sp.to_csv("result_sp.csv",index=False)

In [247]:
merged_sp_dow=pd.merge(result_dow,result_sp,on="Year",suffixes=["_dow","_sp"],how="outer")
merged_sp_dow.to_csv("result_sp_dow.csv",index=False)

In [248]:
result_sp[result_sp["Earn"]>result_sp["Comp_Earn"]].groupby("Presidency").count()

,Year,Earn,Comp_Earn
Presidency,,,
Dem,7,7,7
Rep,7,7,7


In [249]:
len(result_sp[result_sp["Earn"]>0])

19

In [250]:
outperform_sp=len(result_sp[result_sp["Earn"]>result_sp["Comp_Earn"]])
outperform_sp

14

In [251]:
underperform_sp=len(result_sp[result_sp["Earn"]<result_sp["Comp_Earn"]])
underperform_sp

10

In [252]:
df=pd.DataFrame({"dow":[outperform_dow,underperform_dow],"sp":[outperform_sp,underperform_sp]})
df_T=df.T
df_T.columns=["over","under"]
df_T

,over,under
dow,14,12
sp,14,10


In [253]:
df_T.to_csv("over_under.csv",index=True)

In [254]:
result_sp["Earn"].mean()

np.float64(1.9421422298233988)

In [255]:
result_sp["Comp_Earn"].mean()

np.float64(1.0569344834079697)

In [256]:
result_dow["Earn"].mean()

np.float64(2.2900005046213607)

In [257]:
result_dow["Comp_Earn"].mean()

np.float64(0.903508443522656)

In [258]:
df=pd.DataFrame({"sp":[result_sp["Earn"].mean(),result_sp["Comp_Earn"].mean()],
"dow":[result_dow["Earn"].mean(),result_dow["Comp_Earn"].mean()]})
df_T=df.T
df_T.columns=["Wahl","Vergleichswert"]
df_T.to_csv("kursgewinne_schnitt.csv",index=True)

In [259]:
result_sp

,Year,Earn,Comp_Earn,Presidency
0,2020,11.493114,1.636536,Dem
1,2016,6.706115,1.454696,Rep
2,2012,1.260237,1.892105,Dem
3,2008,-8.522613,-0.506033,Dem
4,2004,5.358133,-0.434708,Rep
5,2000,-6.040684,2.326541,Rep
6,1996,6.800283,1.738290,Dem
7,1992,2.786764,1.532893,Dem
8,1988,0.519889,1.968810,Rep
9,1984,0.367943,0.906053,Rep


In [260]:
result_dow.groupby("Presidency")["Earn"].mean()

Presidency
Dem    1.414021
Rep    3.165980
Name: Earn, dtype: float64

In [261]:
result_dow.groupby("Presidency")["Earn"].mean()/result_dow.groupby("Presidency")["Comp_Earn"].mean()

Presidency
Dem    1.908280
Rep    2.969894
dtype: float64

In [262]:
result_sp.groupby("Presidency")["Earn"].mean()

Presidency
Dem    1.29014
Rep    2.71269
Name: Earn, dtype: float64

In [263]:
result_sp.groupby("Presidency")["Comp_Earn"].mean()

Presidency
Dem    0.742384
Rep    1.428676
Name: Comp_Earn, dtype: float64

In [264]:
dow_overperform_pres=result_dow[result_dow["Earn"]>result_dow["Comp_Earn"]].groupby("Presidency")["Year"].count()
dow_overperform_pres

Presidency
Dem    6
Rep    8
Name: Year, dtype: int64

In [265]:
sp_overperform_pres=result_sp[result_sp["Earn"]>result_sp["Comp_Earn"]].groupby("Presidency")["Year"].count()
sp_overperform_pres

Presidency
Dem    7
Rep    7
Name: Year, dtype: int64

In [266]:
pd.DataFrame([dow_overperform_pres,sp_overperform_pres],index=["Dow Jones","S&P 500"]).to_csv("overperform_by_pres.csv",index=True)

In [267]:
len(result_dow[result_dow["Earn"]>result_dow["Comp_Earn"]])-len(result_dow[result_dow["Earn"]<result_dow["Comp_Earn"]])/len(result_dow[result_dow["Earn"]<result_dow["Comp_Earn"]])

13.0

In [268]:
len(result_sp[result_sp["Earn"]>result_sp["Comp_Earn"]])/len(result_sp[result_sp["Earn"]<result_sp["Comp_Earn"]])

1.4

In [269]:
shift_df_sp=(result_sp[result_sp["Presidency"]!=result_sp.shift(-1)["Presidency"]])
print(len(shift_df_sp))
shift_df_sp

12


,Year,Earn,Comp_Earn,Presidency
0,2020,11.493114,1.636536,Dem
1,2016,6.706115,1.454696,Rep
3,2008,-8.522613,-0.506033,Dem
5,2000,-6.040684,2.326541,Rep
7,1992,2.786764,1.532893,Dem
10,1980,4.820917,0.899534,Rep
11,1976,1.170076,-0.229923,Dem
13,1968,1.208786,0.727965,Rep
15,1960,6.212489,0.654440,Dem
17,1952,6.270397,1.688172,Rep


In [270]:
shift_sp_earn=shift_df_sp["Earn"].mean()
shift_sp_earn

np.float64(3.155307972314422)

In [271]:
shift_sp_comp=shift_df_sp["Comp_Earn"].mean()
shift_sp_earn/shift_sp_comp

np.float64(3.3694064307874467)

In [272]:
shift_dow_earn=shift_df_dow["Earn"].mean()

In [273]:
shift_dow_comp=shift_df_dow["Comp_Earn"].mean()
shift_dow_earn/shift_dow_comp

np.float64(3.1099007609634857)

In [274]:
df=pd.DataFrame({"S&P 500":[shift_sp_earn,shift_sp_comp],"Dow Jones":[shift_dow_earn,shift_dow_comp]})
df_T=df.T
df_T.columns=["Wahl","Vergleichszeitraum"]
df_T.to_csv("shift_vergleich.csv")

In [275]:
len(shift_df_sp[shift_df_sp["Earn"]>shift_df_sp["Comp_Earn"]])

10

In [276]:
shift_df_dow=(result_dow[result_dow["Presidency"]!=result_dow.shift(-1)["Presidency"]])
print(len(shift_df_dow))
shift_df_dow

12


,Year,Earn,Comp_Earn,Presidency
23,1928,14.152291,2.936769,Rep
18,1948,-6.092756,0.982512,Dem
16,1956,-0.586767,1.940659,Rep
14,1964,0.619714,1.333410,Dem
12,1972,4.507612,0.123432,Rep
11,1976,0.723321,0.033670,Dem
8,1988,1.101081,2.224007,Rep
6,1996,8.337170,2.153087,Dem
4,2004,5.647904,0.013346,Rep
2,2012,0.864995,1.700283,Dem


In [277]:
len(shift_df_dow[shift_df_dow["Earn"]>shift_df_dow["Comp_Earn"]])

7